In [1]:
!pwd

/panda/eedy/chile/s1_a149/notebook


In [2]:
cp ../mintpy_split_ion_upsample/inputs/ionStack.h5 ../mintpy_split_ion_upsample/inputs/ifgramStack.h5 .

In [8]:
!modify_network.py ionStack.h5 --reset

No lookup table (longitude or rangeCoord) found in files.
--------------------------------------------------
reset dataset 'dropIfgram' to True for all interferograms for file: ionStack.h5
All dropIfgram are already True, no need to reset.


In [9]:
!modify_network.py ionStack.h5 --reference ifgramStack.h5

No lookup table (longitude or rangeCoord) found in files.
open ifgramStack file: ionStack.h5
number of interferograms: 912
--------------------------------------------------
use reference pairs info from file: ifgramStack.h5
number of interferograms in reference: 881
date12 not in reference file: (31)
['20150107_20220331', '20150131_20220331', '20150131_20220424', '20150224_20220331', '20150224_20220424', '20150224_20220506', '20171122_20180614', '20171122_20180626', '20171122_20180708', '20180427_20180813', '20180509_20180813', '20180509_20180825', '20180521_20180825', '20180521_20200802', '20180708_20181012', '20180720_20181012', '20180720_20190516', '20180801_20181012', '20180801_20190516', '20180801_20190528', '20180813_20200802', '20180825_20200802', '20181012_20190516', '20181012_20190528', '20181012_20190609', '20190410_20190808', '20190422_20190808', '20190422_20190820', '20190504_20190808', '20190504_20190820', '20190504_20190901']
---------------------------------------------

In [10]:
!modify_network.py ifgramStack.h5 --reset

No lookup table (longitude or rangeCoord) found in files.
--------------------------------------------------
reset dataset 'dropIfgram' to True for all interferograms for file: ifgramStack.h5
All dropIfgram are already True, no need to reset.


In [11]:
!modify_network.py ifgramStack.h5  --reference ionStack.h5

No lookup table (longitude or rangeCoord) found in files.
open ifgramStack file: ifgramStack.h5
number of interferograms: 881
--------------------------------------------------
use reference pairs info from file: ionStack.h5
number of interferograms in reference: 881
date12 not in reference file: (0)
[]
--------------------------------------------------
number of interferograms to remove: 0
number of interferograms to keep  : 881
Calculated date12 to drop is the same as existing marked input file, skip updating file.


In [14]:
import h5py
import numpy as np

# 读取 ifgramStack.h5 文件
with h5py.File("ifgramStack.h5", "r+") as file:
    # 读取 "/dropIfgram" 数组和 "/unwrapPhase" 数组
    drop_ifgram = file["/dropIfgram"][:]
    unwrap_phase = file["/unwrapPhase"][:]
    date = file["/date"][:]
    bperp = file["/bperp"][:]

    # 确定要保留的数据的索引
    keep_indices = np.where(drop_ifgram == 1)[0]

    # 更新 "/unwrapPhase" 数组，只保留对应 dropIfgram 为 1 的数据
    unwrap_phase = unwrap_phase[keep_indices]
    date = date[keep_indices]
    drop_ifgram = drop_ifgram[keep_indices]
    bperp = bperp[keep_indices]

    # 删除对应 dropIfgram 为 0 的数据
    del file["/unwrapPhase"]
    file.create_dataset("/unwrapPhase", data=unwrap_phase)
    del file["/date"]
    file.create_dataset("/date", data=date)
    del file["/dropIfgram"]
    file.create_dataset("/dropIfgram", data=drop_ifgram)
    del file["/bperp"]
    file.create_dataset("/bperp", data=bperp)

    # 保存修改后的 ifgramStack.h5 文件
    file.flush()

# 读取 ionStack.h5 文件
with h5py.File("ionStack.h5", "r+") as file:
    # 读取 "/dropIfgram" 数组和 "/unwrapPhase" 数组
    drop_ifgram = file["/dropIfgram"][:]
    unwrap_phase = file["/unwrapPhase"][:]
    date = file["/date"][:]
    bperp = file["/bperp"][:]

    # 确定要保留的数据的索引
    keep_indices = np.where(drop_ifgram == 1)[0]

    # 更新 "/unwrapPhase" 数组，只保留对应 dropIfgram 为 1 的数据
    unwrap_phase = unwrap_phase[keep_indices]
    date = date[keep_indices]
    drop_ifgram = drop_ifgram[keep_indices]
    bperp = bperp[keep_indices]

    # 删除对应 dropIfgram 为 0 的数据
    del file["/unwrapPhase"]
    file.create_dataset("/unwrapPhase", data=unwrap_phase)
    del file["/date"]
    file.create_dataset("/date", data=date)
    del file["/dropIfgram"]
    file.create_dataset("/dropIfgram", data=drop_ifgram)
    del file["/bperp"]
    file.create_dataset("/bperp", data=bperp)

    # 保存修改后的 ionStack.h5 文件
    file.flush()

In [15]:
!diff.py ifgramStack.h5 ionStack.h5

ifgramStack.h5 - ['ionStack.h5'] --> ifgramStack_diff_ionStack.h5
the 1st input file is: ifgramStack
open ifgramStack file: ifgramStack.h5
open ifgramStack file: ionStack.h5
reading unwrapPhase from file ifgramStack.h5 ...
reading unwrapPhase from file ionStack.h5 ...
referencing to pixel (541,89) ...
create HDF5 file: ifgramStack_diff_ionStack.h5 with w mode
create dataset /unwrapPhase of float32    in size of (881, 632, 225)      with compression=None
create dataset /bperp       of float32    in size of (881,)     with compression=None
create dataset /date        of |S8        in size of (881, 2)   with compression=None
create dataset /dropIfgram  of bool       in size of (881,)     with compression=None
finished writing to ifgramStack_diff_ionStack.h5
time used: 00 mins 13.6 secs


In [17]:
ionStack_path = 'ionStack.h5'
with h5py.File(ionStack_path, 'r') as file:
    date = file['date']
    date = date[()]
ion_date_list = np.char.decode(date)
ion_date_list = [f'{start_date}_{end_date}' for start_date, end_date in ion_date_list]

In [ ]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from mintpy.view import prep_slice, plot_slice


for i in range(len(ion_date_list)):
    print(i)
    opt = f' --wrap -c cmy --noverbose --zero-mask'
    cmd_list = [f'view.py ifgramStack.h5 unwrap*{ion_date_list[i]} {opt} --title wrap ',
                f'view.py ionStack.h5 *{ion_date_list[i]} {opt} --title ion --ref-yx 548 59', 
                f'view.py ifgramStack_diff_ionStack.h5  *{ion_date_list[i]} {opt} --title correct']

    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=[12, 8])
    for ax, cmd in zip(axs, cmd_list):
        data, atr, inps = prep_slice(cmd)
        plot_slice(ax, data, atr, inps)
    # 保存图片到pic_ion目录下，文件名为i
    plt.savefig(f'pic_ion/{ion_date_list[i]}.png')
    plt.show()
    plt.close()